In [1]:
import sys
cpath = !pwd
#sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')

In [2]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder
from sklearn import preprocessing
from datasets.dataset import Dataset
from lionforests import LionForests

In [8]:
wine_q = Dataset()
X, y, feature_names, class_names = wine_q.load_wine_quality()

In [9]:
y = np.array(y)

In [10]:
LF = LionForests(None, False, None, list(feature_names), class_names=["quality"], categorical_features=None)
parameters = [{
    'max_depth': [10],
    'max_features': [0.75],
    'bootstrap': [True],
    'min_samples_leaf' : [5],
    'n_estimators': [1000]
}]
LF.fit(X, y, params=parameters)

In [11]:
LF.error

-0.5501664539583015

In [18]:
len(y)

6497

In [26]:
LF.explain(X[535], reduction=True, ar_algorithm=None, cl_algorithm=None, save_plots=False, instance_qe=0, clusters=0, method = 'R1')

['if 10.25<=alcohol<=10.3 & 0.215<=volatile acidity<=0.225 & 1.0<=free sulfur dioxide<=4.0 & 0.999<=density<=0.999 & 0.865<=sulphates<=0.89 & 2.05<=residual sugar<=2.15 & 27.5<=total sulfur dioxide<=28.5 & 0.077<=chlorides<=0.079 & 0.235<=citric acid<=0.245 & 3.405<=pH<=3.415 & 9.1<=fixed acidity<=9.15 then quality: 5.9315',
 1000,
 11,
 1000,
 11,
 {'alcohol': [10.25000008046627, 10.30000006854534],
  'volatile acidity': [0.2149999997764826, 0.22500000141561033],
  'free sulfur dioxide': [1.0, 4.000000022351742],
  'density': [0.9989950001458824, 0.9990500002036243],
  'sulphates': [0.865000005364418, 0.8899999994039536],
  'residual sugar': [2.0500000193715096, 2.1500000018626455],
  'total sulfur dioxide': [27.499999474734064, 28.49999967589974],
  'chlorides': [0.07650000014901162, 0.07899999973922969],
  'citric acid': [0.23499999433755872, 0.2450000001490116],
  'pH': [3.4050000190734866, 3.4150000238418583],
  'fixed acidity': [9.099999879300595, 9.149999995529653]},
 None]

In [27]:
LF.visualize(LF.explain(X[535], True, to_vis=True))

LionForests on regression tasks can reduce features and paths through only 3 techniques. 1) Association Rules + Random Selection (R1), 2) Inner Distribution (R2) and 3) Outter Ddistribution (R3)
Setting the default: R1
Prediction rule: if 10.25<=alcohol<=10.3 & 0.215<=volatile acidity<=0.225 & 1.0<=free sulfur dioxide<=4.0 & 0.999<=density<=0.999 & 0.865<=sulphates<=0.89 & 2.05<=residual sugar<=2.15 & 27.5<=total sulfur dioxide<=28.5 & 0.077<=chlorides<=0.079 & 0.235<=citric acid<=0.245 & 3.405<=pH<=3.415 & 9.1<=fixed acidity<=9.15 then quality: 5.9315


Output()

In [26]:
from tqdm import tqdm
import time

results = []
iterr = 100
with tqdm(total=5*4*2*4*5) as pbar:
    for err in [0, 0.01, 0.1, 1, 10]:
        for i in [1, 5, 7, 10]:
            for j in [500, 1000]:#500, 1000
                for o in ['sqrt', 'log2', 0.75, None]:
                    parameters = [{
                        'max_depth': [i],
                        'max_features': [o],
                        'bootstrap': [True],
                        'min_samples_leaf': [5],
                        'n_estimators': [j]
                    }]
                    lf = LionForests(None, False, None, list(feature_names), class_names=["House Price"], categorical_features=None)
                    lf.fit(X, y, params=parameters)
                    tlf = 0
                    fr = 0
                    pr = 0
                    for inde in range(iterr):
                        t_a = time.time()
                        a, b, c, d, e, f, g = lf.explain(
                            X[inde], False, None, None, False)
                        fr = fr + c - e
                        pr = pr + b - d
                        tlf = tlf + time.time() - t_a
                    results.append([str(i), str(j), str(o), 'NoRed','_', '_', tlf/iterr, fr/iterr, pr/iterr, 0, 0])
                    pbar.update(1)
                    error = err*(y.max()-y.min())/100
                    if error == 0:
                        error = abs(lf.error)
                    for k in ['R1', 'R2', 'R3']:
                        if 'R1' == k:
                            for ara in ['apriori', 'fpgrowth']:
                                tlf = 0
                                fr = 0
                                pr = 0
                                lr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, f, g = lf.explain(
                                        X[inde], True, ara,  instance_qe=error, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    if g is not None:
                                        lr = lr + abs(g)
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i), str(j), str(
                                    o), k, ara, '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                                pbar.update(1)
                        else:
                            tlf = 0
                            fr = 0
                            pr = 0
                            lr = 0
                            for inde in range(iterr):
                                t_a = time.time()
                                a, b, c, d, e, f, g = lf.explain(
                                    X[inde], True, None, None,  instance_qe=error, method=k)
                                fr = fr + c - e
                                pr = pr + b - d
                                if g is not None:
                                    lr = lr + abs(g)
                                tlf = tlf + time.time() - t_a
                            results.append([str(i), str(j), str(
                                o), k, '_', '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                            pbar.update(1)

 10%|████████                                                                         | 80/800 [1:03:58<12:10:02, 60.84s/it]C:\Users\iamollas\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
 90%|████████████████████████████████████████████████████████████████████████▉        | 720/800 [9:49:04<1:23:50, 62.89s/it]C:\Users\iamollas\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████████████████████████████████████████████████████████████████████████████| 800/800 [11:14:04<00:00, 50.56s/it]


In [28]:
results

[['1', '500', 'sqrt', 'NoRed', '_', '_', 0.3420617890357971, 0.0, 0.0, 0, 0],
 ['1',
  '500',
  'sqrt',
  'R1',
  'apriori',
  '_',
  0.3438420248031616,
  0.0,
  0.0,
  0.6387553350489675,
  0.0],
 ['1',
  '500',
  'sqrt',
  'R1',
  'fpgrowth',
  '_',
  0.3430867576599121,
  0.0,
  0.0,
  0.6387553350489675,
  0.0],
 ['1',
  '500',
  'sqrt',
  'R2',
  '_',
  '_',
  0.3677923107147217,
  6.26,
  421.42,
  0.6387553350489675,
  0.37545480780982077],
 ['1',
  '500',
  'sqrt',
  'R3',
  '_',
  '_',
  0.36619962930679323,
  0.0,
  0.0,
  0.6387553350489675,
  0.0],
 ['1', '500', 'log2', 'NoRed', '_', '_', 0.34185028791427613, 0.0, 0.0, 0, 0],
 ['1',
  '500',
  'log2',
  'R1',
  'apriori',
  '_',
  0.33928723096847535,
  0.0,
  0.0,
  0.6387553350489675,
  0.0],
 ['1',
  '500',
  'log2',
  'R1',
  'fpgrowth',
  '_',
  0.34323376893997193,
  0.0,
  0.0,
  0.6387553350489675,
  0.0],
 ['1',
  '500',
  'log2',
  'R2',
  '_',
  '_',
  0.3678408050537109,
  6.26,
  421.42,
  0.6387553350489675,
